In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from normal_evaluation.drbart_evaluation import *

In [2]:
model_name = 'artificial'
log_name = 'train'
with open('../transformed_event_logs/artificial_start_end_2_'+log_name+'.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

#test_event_log['time:timestamp'] = test_event_log['time:timestamp_complete']
test_event_log['case:concept:name'] = test_event_log['case:concept:name'].astype(str)
known_resources = ['1', 'Clark', 'Jane', 'Joe', 'Karsten']
known_activities = ['DIAGNOSIS', 'QUALITY_CONTROL', 'REPAIR']

In [3]:
test_event_log

,concept:name,lifecycle:transition_start,time:timestamp_start,org:resource,case:concept:name,id_start,lifecycle:transition_complete,time:timestamp_complete,id_complete,duration,duration_seconds,seconds_in_day,DIAGNOSIS,QUALITY_CONTROL,REPAIR,1,Clark,Jane,Joe,Karsten
0,DIAGNOSIS,START,2020-01-01 03:57:40.044121+00:00,Jane,0,1,COMPLETE,2020-01-01 04:34:23.549454+00:00,2,0 days 00:36:43.505333,2203,14260,1,0,0,0,0,1,0,0
1,REPAIR,START,2020-01-01 04:34:23.549454+00:00,Joe,0,4,COMPLETE,2020-01-01 14:30:27.423999+00:00,5,0 days 09:56:03.874545,35763,16463,1,0,1,0,0,1,1,0
3,DIAGNOSIS,START,2020-01-01 08:16:35.844753+00:00,Jane,1,10,COMPLETE,2020-01-01 08:47:14.772217+00:00,11,0 days 00:30:38.927464,1838,29795,1,0,0,0,0,1,0,0
4,REPAIR,START,2020-01-01 08:47:14.772217+00:00,Karsten,1,13,COMPLETE,2020-01-01 13:27:22.316694+00:00,14,0 days 04:40:07.544477,16807,31634,1,0,1,0,0,1,0,1
5,QUALITY_CONTROL,START,2020-01-01 13:27:22.316694+00:00,Clark,1,16,COMPLETE,2020-01-01 17:21:42.825766+00:00,17,0 days 03:54:20.509072,14060,48442,1,1,1,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5400,DIAGNOSIS,START,2024-12-26 18:05:59.540931+00:00,Clark,1800,16201,COMPLETE,2024-12-26 19:13:26.892809+00:00,16202,0 days 01:07:27.351878,4047,65159,1,0,0,0,1,0,0,0
5399,QUALITY_CONTROL,START,2024-12-26 19:03:41.622915+00:00,Joe,1799,16198,COMPLETE,2024-12-26 21:39:17.312456+00:00,16199,0 days 02:35:35.689541,9335,68621,1,1,1,1,0,0,1,1
5401,REPAIR,START,2024-12-26 19:13:26.892809+00:00,Clark,1800,16204,COMPLETE,2024-12-27 08:23:46.946760+00:00,16205,0 days 13:10:20.053951,47420,69206,1,0,1,0,2,0,0,0
5402,QUALITY_CONTROL,START,2024-12-27 08:23:46.946760+00:00,Jane,1800,16207,COMPLETE,2024-12-27 11:10:44.009123+00:00,16208,0 days 02:46:57.062363,10017,30226,1,1,1,0,2,1,0,0


In [4]:
N = 10000
n_processes = 10
import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

In [5]:
drbart_model_A = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name/',
                     strict_parser=True)
evaluator_A = conduct_evaluation.ConductEvaluation(drbart_model_A, SampleOutcomes_DRBART_Normal_A,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A = evaluator_A.sample_cases(False, True)

100%|██████████████████████████████████████████████████████████████████████████████| 1802/1802 [00:53<00:00, 33.81it/s]


  0%|                                                                       | 2/1802 [05:18<65:55:15, 131.84s/it]

  0%|▎                                                                       | 7/1802 [05:30<13:36:30, 27.29s/it]

  1%|▋                                                                       | 16/1802 [05:32<4:28:06,  9.01s/it]

  1%|▋                                                                       | 18/1802 [05:34<3:45:35,  7.59s/it]

...

 99%|█████████████████████████████████████████████████████████████████████ | 1777/1802 [2:02:51<00:13,  1.92it/s]

 99%|█████████████████████████████████████████████████████████████████████▍| 1787/1802 [2:02:51<00:04,  3.51it/s]

 99%|█████████████████████████████████████████████████████████████████████▌| 1790/1802 [2:02:53<00:04,  2.72it/s]

100%|█████████████████████████████████████████████████████████████████████▉| 1801/1802 [2:02:54<00:00,  4.32it/s]

100%|██████████████████████████████████████████████████████████████████████| 1802/1802 [2:02:54<00:00,  4.09s/it]

In [6]:
np.mean([v.ln() for v in likelihoods_A[0].values()])

Decimal('0.1321764542236615570438066290')

In [7]:
np.mean(get_pscores(likelihoods_A))

np.float64(11021.368005449533)

In [8]:
drbart_model_A_R = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource/',
                     strict_parser=True)
evaluator_A_R = conduct_evaluation.ConductEvaluation(drbart_model_A_R, SampleOutcomes_DRBART_Normal_A_R,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R = evaluator_A_R.sample_cases(False, True)

100%|██████████████████████████████████████████████████████████████████████████████| 1802/1802 [00:53<00:00, 33.93it/s]


  0%|                                                                       | 2/1802 [06:47<87:42:03, 175.40s/it]

  1%|▍                                                                      | 10/1802 [06:51<11:31:05, 23.14s/it]

  1%|▍                                                                      | 11/1802 [06:59<10:29:37, 21.09s/it]

  1%|▌                                                                       | 14/1802 [07:03<6:53:07, 13.86s/it]

...

 99%|█████████████████████████████████████████████████████████████████████▍| 1786/1802 [1:44:50<00:05,  3.03it/s]

 99%|█████████████████████████████████████████████████████████████████████▍| 1788/1802 [1:44:52<00:05,  2.47it/s]

100%|█████████████████████████████████████████████████████████████████████▊| 1796/1802 [1:44:52<00:01,  4.01it/s]

100%|█████████████████████████████████████████████████████████████████████▉| 1799/1802 [1:44:54<00:00,  3.12it/s]

100%|██████████████████████████████████████████████████████████████████████| 1802/1802 [1:44:54<00:00,  3.49s/it]

In [9]:
np.mean([v.ln() for v in likelihoods_A_R[0].values()])

Decimal('0.2626011583163877220596528544')

In [10]:
np.mean(get_pscores(likelihoods_A_R))

np.float64(9802.731507101133)

In [ ]:
drbart_model_R_S = DRBART(parser_dir = '../../../models/'+model_name+'/resource_seconds-in-day/',
                     strict_parser=True)
evaluator_R_S = conduct_evaluation.ConductEvaluation(drbart_model_R_S, SampleOutcomes_DRBART_Normal_R_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_S = evaluator_R_S.sample_cases(False, True)

 13%|██████████▍                                                                    | 238/1802 [03:10<20:50,  1.25it/s]Process ForkPoolWorker-247:
Process ForkPoolWorker-204:
Process ForkPoolWorker-232:
Process ForkPoolWorker-237:
Process ForkPoolWorker-243:
Process ForkPoolWorker-205:
Process ForkPoolWorker-209:
Process ForkPoolWorker-229:
Process ForkPoolWorker-220:
Process ForkPoolWorker-242:
Process ForkPoolWorker-206:
Process ForkPoolWorker-228:
Process ForkPoolWorker-207:
Process ForkPoolWorker-201:
Process ForkPoolWorker-202:
Process ForkPoolWorker-244:
Process ForkPoolWorker-236:
Process ForkPoolWorker-208:
Process ForkPoolWorker-213:
Process ForkPoolWorker-239:
Process ForkPoolWorker-249:
Process ForkPoolWorker-238:
Process ForkPoolWorker-246:
Process ForkPoolWorker-226:
Process ForkPoolWorker-225:
Process ForkPoolWorker-227:
Process ForkPoolWorker-211:
Process ForkPoolWorker-215:
Process ForkPoolWorker-231:
Process ForkPoolWorker-223:
Process ForkPoolWorker-250:
Process ForkP

  0%|                                                                        | 2/1802 [03:54<48:27:53, 96.93s/it]

  0%|▏                                                                       | 5/1802 [03:58<14:09:43, 28.37s/it]

  0%|▎                                                                        | 7/1802 [04:02<8:48:05, 17.65s/it]

  1%|▌                                                                       | 14/1802 [04:07<3:07:52,  6.30s/it]

...

 98%|████████████████████████████████████████████████████████████████████▊ | 1770/1802 [1:58:48<00:27,  1.18it/s]

 98%|████████████████████████████████████████████████████████████████████▊ | 1773/1802 [1:58:50<00:24,  1.18it/s]

 99%|█████████████████████████████████████████████████████████████████████▌| 1790/1802 [1:58:50<00:03,  3.47it/s]

100%|█████████████████████████████████████████████████████████████████████▋| 1795/1802 [1:58:56<00:03,  2.13it/s]

100%|██████████████████████████████████████████████████████████████████████| 1802/1802 [1:58:56<00:00,  3.96s/it]

In [ ]:
np.mean([v.ln() for v in likelihoods_R_S[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_R_S))

In [ ]:
drbart_model_R = DRBART(parser_dir = '../../../models/'+model_name+'/resource/',
                     strict_parser=True)
evaluator_R = conduct_evaluation.ConductEvaluation(drbart_model_R, SampleOutcomes_DRBART_Normal_R,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R = evaluator_R.sample_cases(False, True)

  0%|                                                                      | 2/1802 [08:42<108:28:08, 216.94s/it]

  0%|▏                                                                       | 6/1802 [09:05<27:01:31, 54.17s/it]

  0%|▎                                                                       | 7/1802 [09:25<23:17:37, 46.72s/it]

  1%|▌                                                                       | 14/1802 [09:39<8:00:12, 16.11s/it]

  1%|▋                                                                       | 16/1802 [09:45<6:34:26, 13.25s/it]

...

 99%|█████████████████████████████████████████████████████████████████████▏| 1780/1802 [2:27:35<00:41,  1.88s/it]

 99%|█████████████████████████████████████████████████████████████████████▎| 1783/1802 [2:27:38<00:30,  1.58s/it]

 99%|█████████████████████████████████████████████████████████████████████▎| 1784/1802 [2:27:44<00:37,  2.06s/it]

100%|█████████████████████████████████████████████████████████████████████▊| 1798/1802 [2:27:57<00:05,  1.30s/it]

100%|██████████████████████████████████████████████████████████████████████| 1802/1802 [2:27:57<00:00,  4.93s/it]

In [ ]:
np.mean([v.ln() for v in likelihoods_R[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_R))

In [ ]:
drbart_model_A_R_S = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day/',
                     strict_parser=True)
evaluator_A_R_S = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S, SampleOutcomes_DRBART_Normal_A_R_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S = evaluator_A_R_S.sample_cases(False, True)

  0%|                                                                        | 2/1802 [02:28<32:05:22, 64.18s/it]

  0%|▏                                                                       | 4/1802 [02:29<12:07:39, 24.28s/it]

  0%|▎                                                                        | 7/1802 [02:35<5:44:20, 11.51s/it]

  1%|▌                                                                       | 14/1802 [02:41<2:14:34,  4.52s/it]

  1%|▋                                                                       | 18/1802 [02:46<1:39:36,  3.35s/it]

...

100%|█████████████████████████████████████████████████████████████████████▋| 1795/1802 [1:02:34<00:01,  3.80it/s]

100%|█████████████████████████████████████████████████████████████████████▊| 1797/1802 [1:02:35<00:01,  3.35it/s]

100%|█████████████████████████████████████████████████████████████████████▉| 1800/1802 [1:02:35<00:00,  4.10it/s]

100%|█████████████████████████████████████████████████████████████████████▉| 1801/1802 [1:02:35<00:00,  4.21it/s]

100%|██████████████████████████████████████████████████████████████████████| 1802/1802 [1:02:35<00:00,  2.08s/it]

In [ ]:
np.mean([v.ln() for v in likelihoods_A_R_S[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A_R_S))

In [ ]:
drbart_model_A_R_S_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_activity-count/',
                     strict_parser=True)
evaluator_A_R_S_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_AC, SampleOutcomes_DRBART_Normal_A_R_S_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S_AC = evaluator_A_R_S_AC.sample_cases(False, True)

  0%|▏                                                                        | 6/1802 [02:17<8:30:01, 17.04s/it]

  1%|▍                                                                       | 12/1802 [02:18<3:26:14,  6.91s/it]

  1%|▌                                                                       | 15/1802 [02:20<2:29:26,  5.02s/it]

  1%|▋                                                                       | 18/1802 [02:23<1:51:51,  3.76s/it]

  1%|▊                                                                       | 21/1802 [02:26<1:26:45,  2.92s/it]

...

 99%|███████████████████████████████████████████████████████████████████████▍| 1787/1802 [57:40<00:04,  3.06it/s]

 99%|███████████████████████████████████████████████████████████████████████▍| 1789/1802 [57:42<00:04,  2.75it/s]

100%|███████████████████████████████████████████████████████████████████████▋| 1795/1802 [57:42<00:01,  3.74it/s]

100%|███████████████████████████████████████████████████████████████████████▉| 1801/1802 [57:43<00:00,  4.07it/s]

100%|████████████████████████████████████████████████████████████████████████| 1802/1802 [57:43<00:00,  1.92s/it]

In [ ]:
np.mean([v.ln() for v in likelihoods_A_R_S_AC[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A_R_S_AC))

In [ ]:
drbart_model_A_R_S_RC_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_activity-count_resource-count/',
                     strict_parser=True)
evaluator_A_R_S_RC_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_RC_AC, SampleOutcomes_DRBART_Normal_A_R_S_RC_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_resources' : known_resources,
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S_RC_AC = evaluator_A_R_S_RC_AC.sample_cases(False, True)

  0%|▎                                                                        | 9/1802 [02:17<5:33:17, 11.15s/it]

  1%|▍                                                                       | 11/1802 [02:17<4:14:01,  8.51s/it]

  1%|▌                                                                       | 13/1802 [02:20<3:18:14,  6.65s/it]

  1%|▌                                                                       | 15/1802 [02:27<2:52:53,  5.80s/it]

  1%|▉                                                                       | 22/1802 [02:28<1:15:02,  2.53s/it]

...

 99%|███████████████████████████████████████████████████████████████████████▍| 1787/1802 [52:03<00:06,  2.18it/s]

100%|███████████████████████████████████████████████████████████████████████▊| 1796/1802 [52:03<00:01,  3.91it/s]

100%|███████████████████████████████████████████████████████████████████████▊| 1798/1802 [52:03<00:01,  3.95it/s]

100%|███████████████████████████████████████████████████████████████████████▉| 1800/1802 [52:04<00:00,  3.91it/s]

100%|████████████████████████████████████████████████████████████████████████| 1802/1802 [52:04<00:00,  1.73s/it]

In [ ]:
np.mean([v.ln() for v in likelihoods_A_R_S_RC_AC[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A_R_S_RC_AC))

In [ ]:
drbart_model_A_R_S_RC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_resource-count/',
                     strict_parser=True)
evaluator_A_R_S_RC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_RC, SampleOutcomes_DRBART_Normal_A_R_S_RC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_resources' : known_resources,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S_RC = evaluator_A_R_S_RC.sample_cases(False, True)

  0%|▏                                                                        | 5/1802 [01:50<8:28:35, 16.98s/it]

  1%|▍                                                                       | 12/1802 [01:58<3:06:05,  6.24s/it]

  1%|▋                                                                       | 16/1802 [01:59<2:02:52,  4.13s/it]

  1%|▋                                                                       | 18/1802 [02:01<1:41:00,  3.40s/it]

  1%|▊                                                                       | 20/1802 [02:06<1:36:38,  3.25s/it]

...

 99%|███████████████████████████████████████████████████████████████████████▌| 1792/1802 [50:14<00:02,  4.47it/s]

100%|███████████████████████████████████████████████████████████████████████▋| 1794/1802 [50:15<00:01,  4.05it/s]

100%|███████████████████████████████████████████████████████████████████████▊| 1796/1802 [50:16<00:01,  3.40it/s]

100%|███████████████████████████████████████████████████████████████████████▉| 1799/1802 [50:17<00:00,  3.19it/s]

100%|████████████████████████████████████████████████████████████████████████| 1802/1802 [50:17<00:00,  1.67s/it]

In [ ]:
np.mean([v.ln() for v in likelihoods_A_R_S_RC[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A_R_S_RC))

In [ ]:
results = {
    'drbart_model_A' : likelihoods_A,
    'drbart_model_A_R' : likelihoods_A_R,
    'drbart_model_R' : likelihoods_R,
    'drbart_model_R_S' : likelihoods_R_S,
    'drbart_model_A_R_S' : likelihoods_A_R_S,
    'drbart_model_A_R_S_AC' : likelihoods_A_R_S_AC,
    'drbart_model_A_R_S_RC' : likelihoods_A_R_S_RC,
    'drbart_model_A_R_S_RC_AC' : likelihoods_A_R_S_RC_AC
}

with open('./'+model_name+'_dr_bart_evaluation_'+log_name+'.pickle', 'wb') as handle:
    pickle.dump(results, handle)